# Train YOLO
This notebook shows how to train a YOLO model on a custom dataset. The model is trained using one of the datasets available in the project


In [13]:
# import torch
import pandas as pd

# # Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5x')


In [14]:
data = pd.read_csv("./normalized_data/unique_plates.csv")
sample = data.sample(1)

# paths = sample["img_path"].tolist()

# for path in paths:
#     results = model(path)
#     results.show()


In [15]:
sample = data.copy() #data.sample(1000, random_state=123)

percent = 0.9

train = sample.iloc[:int(len(sample)*percent)]
val = sample.iloc[int(len(sample)*percent):]


In [16]:
import os

os.system("rm -rf ./yolo/data")

def copy(src_file, dest): return os.system(f"cp {src_file} {dest}")

os.system("mkdir -p ./yolo/data/images/train")
os.system("mkdir -p ./yolo/data/images/val")

os.system("mkdir -p ./yolo/data/labels/train")
os.system("mkdir -p ./yolo/data/labels/val")

for i in train.index:
    path = train["img_path"][i]

    name = path.split('/')[-1].replace(".jpg", "")

    copy(path, "./yolo/data/images/train/"+name+".jpg")

    with open("./yolo/data/labels/train/" + name+'.txt', 'w') as f:
        f.write("0 ")

        x_center = (train["xmin"][i] + train["xmax"][i]) / 2 / 480
        y_center = (train["ymin"][i] + train["ymax"][i]) / 2 / 480
        width = (train["xmax"][i] - train["xmin"][i]) / 480
        height = (train["ymax"][i] - train["ymin"][i]) / 480

        f.write(str(x_center) + " ")
        f.write(str(y_center) + " ")
        f.write(str(width) + " ")
        f.write(str(height))

for i in val.index:
    path = val["img_path"][i]

    name = path.split('/')[-1]

    copy(path, "./yolo/data/images/val/"+name+".jpg")

    with open("./yolo/data/labels/val/" + name+'.txt', 'w') as f:
        f.write("0 ")
        
        x_center = (val["xmin"][i] + val["xmax"][i]) / 2 / 480
        y_center = (val["ymin"][i] + val["ymax"][i]) / 2 / 480
        width = (val["xmax"][i] - val["xmin"][i]) / 480
        height = (val["ymax"][i] - val["ymin"][i]) / 480

        f.write(str(x_center) + " ")
        f.write(str(y_center) + " ")
        f.write(str(width) + " ")
        f.write(str(height))
        


In [17]:
# python train.py --img 640 --batch 16 --epochs 70 --data coco128.yaml --weights yolov5x.pt --cache
